In [1]:
%pip install -r Requirements.txt

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tranformers (from versions: none)
ERROR: No matching distribution found for tranformers


In [ ]:
import pandas as pd
from transformers import AutoModel, BertTokenizerFast

# Load Dataset
true_data = pd.read_csv(r'Datasets\Fake_or_Real_News\True.csv')
fake_data = pd.read_csv(r'Datasets\Fake_or_Real_News\Fake.csv')

# Generate labels True/Fake under new Target Column in 'true_data' and 'fake_data'
true_data['Target'] = ['True']*len(true_data)
fake_data['Target'] = ['False']*len(fake_data)


# Merge 'true_data' and 'fake_data', by random mixing into a single df called 'data'
df = pd.concat([true_data, fake_data]).sample(frac=1).reset_index(drop=True)


# See how the data looks like
print(df.shape)
df.head()

(44898, 5)


,title,text,subject,date,Target
0,Factbox: Trump on Twitter (July 24) - Chuck Sc...,The following statements were posted to the ve...,politicsNews,"July 25, 2017",True
1,Trump shows his presidential bid is no mere pu...,"MANCHESTER, N.H. (Reuters) - Donald Trump serv...",politicsNews,"February 10, 2016",True
2,Clinton Campaign Attack Dog Warns Trump And Sa...,"Hey, everything is fair game in this war to wi...",politics,"Jan 11, 2016",False
3,FULL SPEECH: Trump Rocks South Carolina With A...,"A record crowd In Anderson, SC last night for ...",politics,"Oct 20, 2015",False
4,U.S. may need new law to address Russian ad bu...,WASHINGTON/SAN FRANCISCO (Reuters) - U.S. legi...,politicsNews,"September 7, 2017",True


In [ ]:
# Create the transformed DataFrame
df_transformed = pd.DataFrame({
    'instruction': df['title'], 
    'input': 'Is this claim True or False?',
    'output': df['Target']
})

# Save to a new CSV file
output_path = r'Datasets\Fake_or_Real_News\FakeorReal_clean.csv'
df_transformed.to_csv(output_path, index=False)

print(f"Cleaned and transformed CSV saved at: {output_path}")

Cleaned and transformed CSV saved at: Datasets\Fake_or_Real_News\FakeorReal_clean.csv


In [ ]:
import pandas as pd

# Load the cleaned CSV file
csv_file_path = r'Datasets\Fake_or_Real_News\FakeorReal_clean.csv'  # Update the path if needed
df = pd.read_csv(csv_file_path)

# Ensure 'output' column is treated as a string
df['output'] = df['output'].astype(str)

# Convert DataFrame to JSON format
json_file_path = r'Datasets\FakeorReal_Alpaca_Test.json'  # Define output JSON file path
df.to_json(json_file_path, orient='records', indent=4, force_ascii=False)

print(f"Converted JSON saved at: {json_file_path}")

Converted JSON saved at: Datasets\FakeorReal_Alpaca_Test.json


In [1]:
import json
from collections import Counter

with open(r'Datasets\FakeorReal_Alpaca_Test.json', 'r', encoding="utf-8") as f:
    datas = json.load(f)

datas = datas[:100]
# Extract labels and count
label_counts = Counter(entry['output'] for entry in datas)
print(label_counts)

Counter({'False': 55, 'True': 45})


In [ ]:
import json

# Input and output file paths
json_file_path = "Datasets\FakeorReal_Alpaca_Test.json"  
jsonl_file_path = "Datasets\FakeorReal_PromptCompletion_Test.jsonl" 

# Load JSON data
with open(json_file_path, mode='r', encoding='utf-8') as json_file:
    data = json.load(json_file) 

# Open the JSONL file for writing
with open(jsonl_file_path, mode='w', encoding='utf-8') as jsonl_file:
    for entry in data:
        # Construct the new format
        json_entry = {
            "prompt": f"'{entry['instruction']}'\n{entry['input']}",
            "completion": entry['output']
        }
        # Write to JSONL file
        jsonl_file.write(json.dumps(json_entry) + "\n")

print(f"Conversion complete! Saved as {jsonl_file_path}")


Conversion complete! Saved as Datasets\FakeorReal_PromptCompletion_Test.jsonl
